### Filtering down to products that have sufficient sales across time period

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
import warnings
import seaborn as sns


## DATA Prep

In [2]:
file_path = 'online_retail_II.xlsx'
xls = pd.ExcelFile(file_path)
df1 = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
df2 = pd.read_excel(xls, sheet_name=xls.sheet_names[1])
retail_df = pd.concat([df1, df2], ignore_index=True)

KeyboardInterrupt: 

In [3]:
retail_df = pd.read_csv("retail_df.csv")
display(retail_df.head())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
# Filter out null descriptions
df_desc_na = retail_df[retail_df['Description'].notna()]

# Filter out the description "Discount" and "Manual"
REMOVE = ["Discount", "Manual"]
df_remove = df_desc_na[~df_desc_na['Description'].isin(REMOVE)]

# Filter our price <= 0
df_no_price = df_remove[df_remove['Price'] > 0]

### Identify products with sales in at least 22 months

In [5]:
filtered_df = df_no_price

filtered_df['InvoiceDate'] = pd.to_datetime(filtered_df['InvoiceDate'])


desc_months = filtered_df.groupby('Description').agg(
    earliest_invoicedate=('InvoiceDate', 'min'),
    latest_invoicedate=('InvoiceDate', 'max'),
    unique_invoice_months=('InvoiceDate', lambda x: x.dt.to_period('M').nunique()),
    sum_quantity=('Quantity', 'sum')
).reset_index()

THRESHOLD = 22

descriptions_month_filtered = desc_months[desc_months['unique_invoice_months'] >= THRESHOLD]

# Display the filtered dataframe
display(descriptions_month_filtered)


/var/folders/60/16v8sbrj54sc699v387gv6j80000gn/T/ipykernel_4425/1191397884.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['InvoiceDate'] = pd.to_datetime(filtered_df['InvoiceDate'])


,Description,earliest_invoicedate,latest_invoicedate,unique_invoice_months,sum_quantity
44,12 DAISY PEGS IN WOOD BOX,2009-12-02 10:00:00,2011-10-16 12:06:00,23,970
45,12 EGG HOUSE PAINTED WOOD,2009-12-01 11:41:00,2011-12-08 15:54:00,25,563
47,12 IVORY ROSE PEG PLACE SETTINGS,2009-12-03 16:54:00,2011-09-21 15:43:00,22,2798
50,12 PENCIL SMALL TUBE WOODLAND,2009-12-06 14:37:00,2011-12-09 10:03:00,22,10057
54,12 PENCILS SMALL TUBE SKULL,2009-12-01 12:47:00,2011-12-09 12:20:00,25,12899
...,...,...,...,...,...
5380,YOU'RE CONFUSING ME METAL SIGN,2009-12-08 15:49:00,2011-12-09 12:00:00,24,8908
5387,ZINC FOLKART SLEIGH BELLS,2010-01-19 12:13:00,2011-12-09 10:26:00,22,8373
5396,ZINC HEART LATTICE T-LIGHT HOLDER,2009-12-14 18:03:00,2011-12-07 09:22:00,25,1471
5400,ZINC METAL HEART DECORATION,2009-12-01 11:41:00,2011-12-08 14:38:00,25,20149


In [6]:
descriptions_month_filtered.to_csv("descriptions_month_filtered.csv", index = False)

In [7]:
# DON'T NEED TO RE-RUN, THIS CODE WAS TO GET THE FILTERED DATA WITH ONLY THOSE PRODUCTS THAT HAVE DATA FOR >=25 MONTHS

df = pd.read_csv("descriptions_month_filtered.csv")

df_filtered = df[df["unique_invoice_months"] >= 25]

df_filtered.to_csv("descriptions_month_filtered_25.csv", index=False)

In [8]:
good_products = pd.read_csv("descriptions_month_filtered_25.csv")
display(good_products)

,Description,earliest_invoicedate,latest_invoicedate,unique_invoice_months,sum_quantity
0,12 EGG HOUSE PAINTED WOOD,2009-12-01 11:41:00,2011-12-08 15:54:00,25,563
1,12 PENCILS SMALL TUBE SKULL,2009-12-01 12:47:00,2011-12-09 12:20:00,25,12899
2,12 PENCILS TALL TUBE SKULLS,2009-12-01 11:37:00,2011-12-09 10:03:00,25,4899
3,12 PENCILS TALL TUBE WOODLAND,2009-12-01 12:47:00,2011-12-09 08:59:00,25,4703
4,12 RED ROSE PEG PLACE SETTINGS,2009-12-03 16:54:00,2011-12-08 19:28:00,25,1277
...,...,...,...,...,...
543,WOVEN ROSE GARDEN CUSHION COVER,2009-12-03 13:32:00,2011-12-09 10:03:00,25,494
544,WRAP PINK FAIRY CAKES,2009-12-01 18:35:00,2011-12-08 12:29:00,25,16615
545,ZINC HEART LATTICE T-LIGHT HOLDER,2009-12-14 18:03:00,2011-12-07 09:22:00,25,1471
546,ZINC METAL HEART DECORATION,2009-12-01 11:41:00,2011-12-08 14:38:00,25,20149
